In [10]:
from typing import Dict

import numpy as np
import random
import tqdm
import functools

In [11]:
NUM_DICE = 2
MAX_DICE_FACE = 6

action_to_index = {}
index_to_action = {}

# Generate action map.
index = 0
for dice_num in range(1, 2 * NUM_DICE + 1):
    for face in range(1, MAX_DICE_FACE + 1):
        action = (dice_num, face)
        action_to_index[action] = index
        index_to_action[index] = action
        index += 1

action_to_index["LIAR"] = index
index_to_action[index] = "LIAR"
non_liar_actions = []

for action in action_to_index.keys():
    if action != "LIAR":
        non_liar_actions.append(action)

non_liar_actions = sorted(non_liar_actions)

NUM_ACTIONS = len(action_to_index)

@functools.lru_cache(maxsize=None)
def allowed_actions(last_bet):
    if last_bet == None:
        return non_liar_actions

    if last_bet == "LIAR":
        return []

    last_num, last_face = last_bet

    # Can either increase number of dice or change the face.
    actions = []

    for action in non_liar_actions:
        action_num, action_face = action

        if action_face > last_face:
            actions.append(action)
            continue

        if action_num > last_num and action_face == last_face:
            actions.append(action)
            continue

    actions.append("LIAR")

    return actions

allowed_actions((1, 1))

[(1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (3, 6),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (4, 6),
 'LIAR']

In [12]:
def sample_action(strategy, valid_actions):
    return valid_actions[np.random.choice(list(range(len(valid_actions))), p=strategy)]

class Node(object):
    """Store the current game state.
    """

    def __init__(self, deciding_player=-1, history=None, deal=False, player_dice=None):
        self._history = history if history is not None else []
        self._player_dice = player_dice
        self._deciding_player = deciding_player
        self._deal = deal

    @property
    def deciding_player_dice(self):
        return self._player_dice[self._deciding_player]

    @property
    def deciding_player(self):
        return self._deciding_player

    @property
    def chance_node(self):
        return self._deal

    @property
    def history(self):
        return self._history

    def sample_chance(self):
        assert self.chance_node
        player_dice = np.random.randint(1, MAX_DICE_FACE + 1, size=(2, NUM_DICE)).tolist()
        player_dice = tuple([tuple(sorted(dice)) for dice in player_dice])

        return Node(
            deciding_player=0,
            history=[],
            deal=False,
            player_dice=player_dice,
        )

    def terminal(self, perspective_player):
        """Return (is_terminal, utility).
        """

        if len(self._history) == 0:
            return False, None

        if len(self._history) > 20:
            return True, 0

        if self._history[-1] == "LIAR":
            # Terminal.
            # Find out who one.
            bet_in_question = self._history[-2]
            bet_num, bet_face = bet_in_question
            player_in_question = self._deciding_player

            actual_num = 0

            for player_index in range(2):
                for dice_index in range(NUM_DICE):
                    if self._player_dice[player_index][dice_index] == bet_face:
                        actual_num += 1

            if actual_num >= bet_num:
                # Player in question wins.
                if player_in_question == perspective_player:
                    return True, 1
                else:
                    return True, -1
            else:
                # Player in question loses.
                if player_in_question == perspective_player:
                    return True, -1
                else:
                    return True, 1

        return False, None

    def take_action(self, action):
        # Assumes action is valid.
        assert not self.chance_node
        return Node(
            deciding_player=1 - self._deciding_player,
            history=self._history + [action],
            deal=False,
            player_dice=self._player_dice,
        )

class InfoSet(object):
    def __init__(self, num_actions):
        self.regret_sum = np.zeros(num_actions)
        self.strategy = np.zeros(num_actions)
        self.strategy_sum = np.zeros(num_actions)
        self.num_actions = num_actions

    def get_strategy(self):
        regret_sum_clipped = np.clip(self.regret_sum, a_min=0, a_max=None)
        denom = np.sum(regret_sum_clipped)

        if denom < 0.001:
            self.strategy = np.ones(self.num_actions) / self.num_actions
        else:
            self.strategy = regret_sum_clipped / denom

        return self.strategy

    def get_average_strategy(self):
        avg_strategy = np.zeros(self.num_actions)
        normalizing_sum = 0
        
        for a in range(self.num_actions):
            normalizing_sum += self.strategy_sum[a]
        for a in range(self.num_actions):
            if normalizing_sum > 0:
                avg_strategy[a] = self.strategy_sum[a] / normalizing_sum
            else:
                avg_strategy[a] = 1.0 / self.num_actions
        
        return avg_strategy


def mccfr_iteration(node: Node, traversing_player: int, infosets: Dict[int, InfoSet]):
    if node.chance_node:
        return mccfr_iteration(node.sample_chance(), traversing_player, infosets)

    is_terminal, utility = node.terminal(traversing_player)
    if is_terminal:
        return utility

    if len(node.history) == 0:
        # First action.
        valid_actions = allowed_actions(None)
    else:
        valid_actions = allowed_actions(node.history[-1])

    info_set_key = (node.deciding_player, node.deciding_player_dice, tuple(node.history))
    if info_set_key not in infosets:
        infosets[info_set_key] = InfoSet(len(valid_actions))
    infoset = infosets[info_set_key]

    if node.deciding_player == traversing_player:
        util = np.zeros(len(valid_actions))
        infoset_util = 0
        strategy = infoset.get_strategy()

        for i, action in enumerate(valid_actions):
            next_node = node.take_action(action)
            util[i] = mccfr_iteration(next_node, traversing_player, infosets)
            infoset_util += strategy[i] * util[i]

        regret = util - infoset_util
        infoset.regret_sum += regret

        return infoset_util

    strategy = infoset.get_strategy()
    action = sample_action(strategy, valid_actions)
    next_node = node.take_action(action)
    util = mccfr_iteration(next_node, traversing_player, infosets)
    infoset.strategy_sum += strategy
    return util

def mccfr(iterations):
    infosets = {}

    util = np.zeros(NUM_ACTIONS)
    for t in tqdm.trange(1, iterations + 1): 
        for traversing_player in range(2):
            node = Node(deal=True)
            util[traversing_player] += mccfr_iteration(node, traversing_player, infosets)

    print('Average game value: {}'.format(util[0]/(iterations)))
    return infosets

infosets = mccfr(10000)

  3%|▎         | 346/10000 [00:13<06:24, 25.11it/s]


KeyboardInterrupt: 

In [118]:
root_chance_node = Node(deal=True)
current_node = root_chance_node.sample_chance()
print("Your dice are: {}".format(current_node.deciding_player_dice))

while True:
    valid_actions = allowed_actions(current_node.history[-1] if len(current_node.history) else None)

    while True:
        try:
            raw_user_move = input("Enter your move: ")
            if "liar" in raw_user_move.lower():
                user_move = "LIAR"
            else:
                user_move = (int(raw_user_move.split(" ")[0]), int(raw_user_move.split(" ")[1]))

            if user_move not in valid_actions:
                print("%s is not a valid move." % user_move)
                continue

            break
        except:
            print("Invalid move: %s. Try again." % raw_user_move)

    print("User move", user_move)
    current_node = current_node.take_action(user_move)

    term, util = current_node.terminal(0)
    if term:
        print(current_node._player_dice)
        print("End game", util)
        break

    valid_actions = allowed_actions(current_node.history[-1] if len(current_node.history) else None)
    bot_strategy = infosets[(current_node.deciding_player, current_node.deciding_player_dice, tuple(current_node.history))].get_average_strategy()
    bot_move = sample_action(bot_strategy, valid_actions)
    print("Bot move", bot_move)
    current_node = current_node.take_action(bot_move)

    term, util = current_node.terminal(0)
    if term:
        print(current_node._player_dice)
        print("End game", util)
        break

Your dice are: (1,)
User move (1, 1)
Bot move (1, 4)
Invalid move: . Try again.
User move LIAR
((1,), (4,))
End game -1


In [8]:
def play_against_random():
    root_chance_node = Node(deal=True)
    current_node = root_chance_node.sample_chance()
    turn = "BOT"

    while True:
        term, util = current_node.terminal(0)

        if term:
            return util

        valid_actions = allowed_actions(current_node.history[-1] if len(current_node.history) else None)

        if turn == "BOT":
            k = (current_node.deciding_player, current_node.deciding_player_dice, tuple(current_node.history))
            if k in infosets:
                bot_strategy = infosets[k].get_average_strategy()
            else:
                bot_strategy = np.ones(len(valid_actions)) / len(valid_actions)

            bot_move = sample_action(bot_strategy, valid_actions)
            current_node = current_node.take_action(bot_move)

            turn = "RAND"
        else:
            rand_strat = np.ones(len(valid_actions)) / len(valid_actions)
            rand_move = sample_action(rand_strat, valid_actions)

            current_node = current_node.take_action(rand_move)
            turn = "BOT"

utility_sum = 0
games = 10000

for _ in tqdm.trange(games):
    utility_sum += play_against_random()

print(f"Average Utility: {utility_sum / games:.2f}")

100%|██████████| 10000/10000 [00:01<00:00, 9348.08it/s]

Average Utility: 0.75


In [9]:
weak_infosets = mccfr(1000)

def play_against_weak():
    root_chance_node = Node(deal=True)
    current_node = root_chance_node.sample_chance()
    turn = "BOT"

    while True:
        term, util = current_node.terminal(0)

        if term:
            return util

        valid_actions = allowed_actions(current_node.history[-1] if len(current_node.history) else None)

        if turn == "BOT":
            k = (current_node.deciding_player, current_node.deciding_player_dice, tuple(current_node.history))
            if k in infosets:
                bot_strategy = infosets[k].get_average_strategy()
            else:
                bot_strategy = np.ones(len(valid_actions)) / len(valid_actions)

            bot_move = sample_action(bot_strategy, valid_actions)
            current_node = current_node.take_action(bot_move)

            turn = "RAND"
        else:
            k = (current_node.deciding_player, current_node.deciding_player_dice, tuple(current_node.history))
            if k in weak_infosets:
                bot_strategy = weak_infosets[k].get_average_strategy()
            else:
                bot_strategy = np.ones(len(valid_actions)) / len(valid_actions)

            bot_move = sample_action(bot_strategy, valid_actions)
            current_node = current_node.take_action(bot_move)

            turn = "BOT"

utility_sum = 0
games = 10000

for _ in tqdm.trange(games):
    utility_sum += play_against_weak()

print(f"Average Utility: {utility_sum / games:.2f}")

100%|██████████| 1000/1000 [00:03<00:00, 277.80it/s]


Average game value: -0.01984133103296207


100%|██████████| 10000/10000 [00:00<00:00, 10019.26it/s]

Average Utility: 0.15
